In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
import plotly
import sqlalchemy
from sqlalchemy import create_engine, text


sb.set_context('talk')

**Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones**

**Exploración y Curación de Datos**

*Edición 2024*

----

# Trabajo práctico entregable - parte 1

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [2]:
melb_df = pd.read_csv(
    './ArchivosCSV/melb_data.csv')

display(melb_df.shape )

melb_df[:3]

(13580, 21)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0


In [3]:
interesting_cols = [
  'description', 'neighborhood_overview',
  'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
  'price', 'weekly_price', 'monthly_price',
  'latitude', 'longitude',
]

airbnb_df = pd.read_csv(
  './ArchivosCSV/cleansed_listings_dec18.csv',
  usecols = interesting_cols
)

airbnb_df[:3]

C:\Users\juanc\AppData\Local\Temp\ipykernel_1652\927775226.py:8: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_df = pd.read_csv(


,description,neighborhood_overview,street,neighborhood,city,suburb,state,zipcode,latitude,longitude,price,weekly_price,monthly_price
0,"House: Clean, New, Modern, Quite, Safe. 10Km f...",Very safe! Family oriented. Older age group.,"Bulleen, VIC, Australia",Balwyn North,Manningham,Bulleen,VIC,3105,-37.772684,145.092133,60,NaN,NaN
1,A large air conditioned room with queen spring...,This hip area is a crossroads between two grea...,"Brunswick East, VIC, Australia",Brunswick,Moreland,Brunswick East,VIC,3057,-37.766505,144.980736,35,200.0,803.0
2,RIGHT IN THE HEART OF ST KILDA! It doesn't get...,A stay at our apartment means you can enjoy so...,"St Kilda, VIC, Australia",St Kilda,Port Phillip,St Kilda,VIC,3182,-37.859755,144.977369,159,1253.0,4452.0


## Ejercicio 1 SQL:

1. Crear una base de datos en SQLite utilizando la libreria SQLalchemy.

2. Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.

3. Implementar consultas en SQL que respondan con la siguiente información:

    - cantidad de registros totales por ciudad.
    - cantidad de registros totales por barrio y ciudad.

4. Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase.  


In [4]:
db_path = 'melb_data.db'
engine = create_engine('sqlite:///' + db_path)

##### BDD de melb_data.csv

In [5]:
melb_df.to_sql(
    'melb_data', #Nombre de la tabla
    con=engine, #Especifica a qué BDD se va a conectar
    if_exists='replace', 
    index=False
)

13580

In [6]:
print(melb_df.columns)

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')


In [15]:
update_query = """
UPDATE melb_data 
SET Postcode = CAST(Postcode AS INTEGER);
"""

with engine.connect() as conn:
    try:
       conn.execute(text(update_query))
       print("La actualizacion se realizo correctamente.")
    except Exception as e:
        print("Se produjo un error durante la actualizacion:", str(e))   

La actualizacion se realizo correctamente.


In [16]:
vercolumna = """
SELECT Postcode
FROM melb_data
"""

df = pd.read_sql_query(vercolumna, con=engine)

print(df)

       Postcode
0        3067.0
1        3067.0
2        3067.0
3        3067.0
4        3067.0
...         ...
13575    3150.0
13576    3016.0
13577    3016.0
13578    3016.0
13579    3013.0

[13580 rows x 1 columns]


##### BDD de AirBNB

In [9]:
airbnb_df.to_sql(
    'airbnb_data', #Nombre de la tabla
    con=engine, #Especifica a qué BDD se va a conectar
    if_exists='replace', #Si ya existe la tabla, se reemplaza con los datos nuevos
    index=False
)

22895

In [10]:
print(airbnb_df.columns)

Index(['description', 'neighborhood_overview', 'street', 'neighborhood',
       'city', 'suburb', 'state', 'zipcode', 'latitude', 'longitude', 'price',
       'weekly_price', 'monthly_price'],
      dtype='object')


In [13]:
query_por_ciudad = """
SELECT city, COUNT(*) AS registros_totales
FROM airbnb_data
GROUP BY city;
"""


df_total_por_ciudad = pd.read_sql_query(
    query_por_ciudad, 
    con=engine
)


print("Cantidad de registros totales por ciudad:")
print(df_total_por_ciudad)

Cantidad de registros totales por ciudad:
                 city  registros_totales
0             Banyule                203
1             Bayside                375
2          Boroondara                664
3            Brimbank                108
4            Cardinia                123
5               Casey                153
6             Darebin                698
7           Frankston                177
8           Glen Eira                631
9   Greater Dandenong                147
10        Hobsons Bay                239
11               Hume                170
12           Kingston                309
13               Knox                175
14         Manningham                313
15        Maribyrnong                436
16          Maroondah                115
17          Melbourne               7368
18             Melton                 95
19             Monash                571
20      Moonee Valley                344
21           Moreland                967
22          Nil

In [14]:
query_por_barrio_ciudad = """
SELECT city, neighborhood, COUNT(*) AS registros_totales
FROM airbnb_data
GROUP BY city, neighborhood;
"""


df_por_barrio_ciudad = pd.read_sql_query(
    query_por_barrio_ciudad, 
    con=engine
)


print("\nCantidad de registros totales por barrio y ciudad:")
print(df_por_barrio_ciudad)


Cantidad de registros totales por barrio y ciudad:
             city   neighborhood  registros_totales
0         Banyule           None                174
1         Banyule     Alphington                 16
2         Banyule        Preston                  8
3         Banyule      Thornbury                  5
4         Bayside           None                170
..            ...            ...                ...
129         Yarra        Fitzroy                381
130         Yarra  Fitzroy North                225
131         Yarra      Northcote                  6
132         Yarra       Richmond                547
133  Yarra Ranges           None                771

[134 rows x 3 columns]


##### JOIN

In [ ]:
#query_join = """
#SELECT *
#FROM melb_data m
#INNER JOIN airbnb_data a ON m.Postcode = a.zipcode;
#"""
#
#
#df_join = pd.read_sql_query(
#    query_join, 
#    con=engine
#)
#
#
#print("\nResultado de la combinación de datasets con JOIN:")
#print(df_join)

## Ejercicio 2 - Pandas:

1. Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar las columnas seleccionadas y las que no lo fueron.
 - Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.


2. Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico.
  1. Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.
  1. Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.
  2. Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

## Ejercicio 3:

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.

## Ejercicios opcionales:

1. Armar un script en python (archivo .py) [ETL](https://towardsdatascience.com/what-to-log-from-python-etl-pipelines-9e0cfe29950e) que corra los pasos de extraccion, transformacion y carga, armando una funcion para cada etapa del proceso y luego un main que corra todos los pasos requeridos.

2. Armar un DAG en Apache Airflow que corra el ETL. (https://airflow.apache.org/docs/apache-airflow/stable/tutorial.html)